In [4]:
import os
from dotenv import load_dotenv
from slack_bolt import App
import pygsheets
import pandas as pd
import re
import logging
import random
from datetime import date

g_client = pygsheets.authorize(service_file='google-credentials.json')

sh = g_client.open('DiversaBot')

sh_spot_history = sh[0]
df_spot_history = sh_spot_history.get_as_df(
    has_header=True,
    index_column=None,
    numerize=False,
    include_tailing_empty=False,
    include_tailing_empty_rows=False
)

In [5]:
def spotter_leaderboard():
    '''
    Returns a leaderboard dataframe for the most spots
    columns = ['COUNT', 'RANK']
    indexed by SPOTTER, NAME
    '''
    global df_spot_history

    df_spot_history = df_spot_history[df_spot_history["FLAGGED"] == "FALSE"]
    counts = df_spot_history.groupby(['SPOTTER', 'NAME']).count()
    counts.rename(columns={'SPOTTED':'COUNT'}, inplace=True)
    counts = counts[['COUNT']]
    counts['RANK'] = counts['COUNT'].rank(ascending=False, method='dense')
    counts.sort_values(by='COUNT', inplace=True, ascending=False)
    return counts

In [7]:
leaderboard = spotter_leaderboard()
leaderboard

,,COUNT,RANK
SPOTTER,NAME,,


In [9]:
leaderboard = leaderboard.reset_index()
leaderboard

,SPOTTER,NAME,COUNT,RANK


In [10]:
len(leaderboard)

0

In [12]:
leaderboard = spotter_leaderboard()
leaderboard = leaderboard.reset_index()
rank = leaderboard.index[leaderboard['SPOTTER']==user].tolist()[0]
size = len(leaderboard)
for i in range(max(0, rank - 4), min(size, rank + 5)):
    row = leaderboard.iloc[i]

NameError: name 'user' is not defined

In [13]:
leaderboard = spotter_leaderboard()
leaderboard = leaderboard.reset_index()
size = len(leaderboard)
for i in range(min(size, 10)):
    row = leaderboard.iloc[i]